In [1]:
pip install faiss-cpu sentence-transformers


   ---------------------------------------- 13.7/13.7 MB 8.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer


C:\Kartik\anaconda3\envs\aidl\lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.14.6 when it was built against 1.14.5, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


In [9]:
model = SentenceTransformer("all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Kartik\anaconda3\envs\aidl\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\argka\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SyntaxError: unterminated string literal (detected at line 1) (1381268170.py, line 1)

In [14]:
from datasets import load_dataset

ds = load_dataset("sentence-transformers/natural-questions")

README.md:   0%|          | 0.00/2.28k [00:00<?, ?B/s]

C:\Kartik\anaconda3\envs\aidl\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\argka\.cache\huggingface\hub\datasets--sentence-transformers--natural-questions. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


train-00000-of-00001.parquet:   0%|          | 0.00/44.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100231 [00:00<?, ? examples/s]

In [13]:
!pip install datasets


   ---------------------------------------- 25.3/25.3 MB 4.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.2.0
    Uninstalling fsspec-2025.2.0:
      Successfully uninstalled fsspec-2025.2.0


In [16]:
ds

DatasetDict({
    train: Dataset({
        features: ['query', 'answer'],
        num_rows: 100231
    })
})

In [18]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
import numpy as np

# Load dataset
ds = load_dataset("sentence-transformers/natural-questions")

# Select a split (e.g., 'train')
train_ds = ds['train']

# Extract the text column (replace 'text' with the actual column name)
text_data = train_ds['query']  # Check dataset features to confirm the column name

# Load model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Convert documents to embeddings
doc_embeddings = model.encode(text_data, convert_to_numpy=True)

# Convert embeddings to FAISS format
doc_embeddings = np.array(doc_embeddings, dtype=np.float32)


In [19]:
# Create a FAISS index (L2 or cosine similarity search)
index = faiss.IndexFlatL2(doc_embeddings.shape[1])  # L2 distance (Euclidean)
index.add(doc_embeddings)  # Add document embeddings to FAISS

print(f"Stored {index.ntotal} documents in FAISS.")


Stored 100231 documents in FAISS.


In [26]:
# Query example
query = "who sang what in the world's come over you?"

# Convert query into embedding
query_embedding = model.encode([query])
query_embedding = np.array(query_embedding, dtype=np.float32)

# Search for the most similar document (top-1 match)
k = 1  # Number of results to return
distances, indices = index.search(query_embedding, k)

distances, indices = index.search(query_embedding, k)
distances, indices = index.search(query_embedding, k)

# Retrieve and print the most relevant document
index_val = int(indices[0][0]) #Convert numpy.int64 to standard int
best_match = ds['train'][index_val]  # Access the 'train' split first
print(f"\n🔍 Query: {query}")
print(f"✅ Best Match: {best_match}")


🔍 Query: who sang what in the world's come over you?
✅ Best Match: {'query': "who sang what in the world's come over you", 'answer': 'Jack Scott (singer) At the beginning of 1960, Scott again changed record labels, this time to Top Rank Records.[1] He then recorded four Billboard Hot 100 hits\xa0– "What in the World\'s Come Over You" (#5), "Burning Bridges" (#3) b/w "Oh Little One" (#34), and "It Only Happened Yesterday" (#38).[1] "What in the World\'s Come Over You" was Scott\'s second gold disc winner.[6] Scott continued to record and perform during the 1960s and 1970s.[1] His song "You\'re Just Gettin\' Better" reached the country charts in 1974.[1] In May 1977, Scott recorded a Peel session for BBC Radio\xa01 disc jockey, John Peel.'}
